# Latihan Membuat Machine Learning Pipeline

In [30]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [31]:
PIPELINE_NAME = "imdb-pipeline"
SCHEMA_PIPELINE_NAME = "imdb-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [32]:
DATA_ROOT = "data"

In [33]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

mebagi dataset yang akan digunakan untuk proses training (“train”) dan evaluasi (“eval”). Pembagian ini dilakukan dengan mengatur parameter output_config dengan rasio 8:2. Rasio ini dapat kita atur menggunakan parameter hash_buckets.

In [34]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Jika ingin melihat komponen ExampleGen secara interaktif, Anda dapat menjalankan komponen tersebut menggunakan object InteractiveContext() yang telah kita definisikan sebelumnya. 

In [35]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [36]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
  
  
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [37]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [38]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [39]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',STRING,required,,'sentiment'


,Values
Domain,
'sentiment',"'negative', 'positive'"


In [40]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [41]:
interactive_context.show(example_validator.outputs['anomalies'])

In [42]:
TRANSFORM_MODULE_FILE = "imdb_transform.py"

In [43]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying imdb_tuner.py -> build/lib
copying imdb_trainer.py -> build/lib
copying imdb_transform.py -> build/lib
installing to /tmp/tmpzc_a7f12
running install
running install_lib
copying build/lib/imdb_tuner.py -> /tmp/tmpzc_a7f12
copying build/lib/imdb_transform.py -> /tmp/tmpzc_a7f12
copying build/lib/imdb_trainer.py -> /tmp/tmpzc_a7f12
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpzc_a7f12/tfx_user_code_Tra

/home/fachri/miniconda3/envs/mlops/lib/python3.8/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/imdb-pipeline/_wheels/tfx_user_code_Transform-0.0+1c3ddf4023be99246a31a7d39ef955798ad51976ebdfcf62b75b06c6d80bea3f-py3-none-any.whl
Processing ./pipelines/imdb-pipeline/_wheels/tfx_user_code_Transform-0.0+1c3ddf4023be99246a31a7d39ef955798ad51976ebdfcf62b75b06c6d80bea3f-py3-none-any.whl
Processing ./pipelines/imdb-pipeline/_wheels/tfx_user_code_Transform-0.0+1c3ddf4023be99246a31a7d39ef955798ad51976ebdfcf62b75b06c6d80bea3f-py3-none-any.whl


INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/c6a88ca61e6d4842937798788327304d/assets


INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/c6a88ca61e6d4842937798788327304d/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [44]:
TUNER_MODULE_FILE = "imdb_tuner.py"

In [45]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2
  
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )
interactive_context.run(tuner)

Trial 5 Complete [00h 00m 30s]
val_binary_accuracy: 0.8467187285423279

Best val_binary_accuracy So Far: 0.8467187285423279
Total elapsed time: 00h 02m 00s
Results summary
Results in pipelines/imdb-pipeline/.temp/6/kt_randomsearch
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
num_hidden_layers: 2
embed_dims: 112
dense_units: 256
learning_rate: 0.01
lstm_units: 96
dropout_rate: 0.2
Score: 0.8467187285423279

Trial 3 summary
Hyperparameters:
num_hidden_layers: 1
embed_dims: 112
dense_units: 224
learning_rate: 0.0001
lstm_units: 32
dropout_rate: 0.1
Score: 0.8462499976158142

Trial 1 summary
Hyperparameters:
num_hidden_layers: 1
embed_dims: 48
dense_units: 160
learning_rate: 0.01
lstm_units: 64
dropout_rate: 0.30000000000000004
Score: 0.8440625071525574

Trial 0 summary
Hyperparameters:
num_hidden_layers: 1
embed_dims: 48
dense_units: 96
learning_rate: 0.001
lstm_units: 128
dropout_rate: 0.30000000000000004
Score: 0.8385937

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [46]:
TRAINER_MODULE_FILE = "imdb_trainer.py"

In [47]:
from tfx.proto import trainer_pb2
  
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
    hyperparameters=tuner.outputs['best_hyperparameters'],
)
interactive_context.run(trainer)

/home/fachri/miniconda3/envs/mlops/lib/python3.8/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying imdb_tuner.py -> build/lib
copying imdb_trainer.py -> build/lib
copying imdb_transform.py -> build/lib
installing to /tmp/tmpsb8u6t8j
running install
running install_lib
copying build/lib/imdb_tuner.py -> /tmp/tmpsb8u6t8j
copying build/lib/imdb_transform.py -> /tmp/tmpsb8u6t8j
copying build/lib/imdb_trainer.py -> /tmp/tmpsb8u6t8j
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpsb8u6t8j/tfx_user_code_Trainer-0.0+1c3ddf4

INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Trainer/model/7/Format-Serving/assets


320/320 [==============================] - 58s 178ms/step - loss: 0.5660 - binary_accuracy: 0.7035 - val_loss: 0.3854 - val_binary_accuracy: 0.8402
Epoch 2/5
320/320 [==============================] - ETA: 0s - loss: 0.3543 - binary_accuracy: 0.8580
Epoch 2: val_loss improved from 0.38543 to 0.32342, saving model to pipelines/imdb-pipeline/Trainer/model/7/Format-Serving
INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Trainer/model/7/Format-Serving/assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/imdb-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [48]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
  
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
  
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [49]:
import tensorflow_model_analysis as tfma 

label_vocabulary = [b'negative', b'positive']  # Define your label vocabulary

label_to_integer = {label: index for index, label in enumerate(label_vocabulary)}

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(
            label_key='sentiment_xf',
            )],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ],
    # Add label vocabulary to the config
    slicing_specs=[
        tfma.SlicingSpec(),
    ]
  
)

In [50]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
  
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [51]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [52]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
  
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/imdb-detection-model'))
  
)
  
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)